# Setup EJECUCIÓN UNICA. 


Import nltk y descargar stopwords de la pestaña de corpus. 

In [1]:

# Descargamos las stopwords desde nltk corpus (Abrira una ventana nueva.)
# Tenemos que ir a la pestaña de CORPUS, buscar la parte de las stopwords y darle descargar.  
if __name__ == '__main__':
    import nltk
    nltk.download()


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Tenemos que importat parser desde tika. Para esto debemos instalar tika usando  
 $pip install tika  
Para la instalación debemos contar con JDK

**NOTA** No es necesario correr la celda de abajo ya que en el proyecto que se envio ya convertimos los pdfs a txt.  
Tampoco es necesario instalar tika a menos que se quiera volver a convertir los pdf en txt. 


In [2]:


if __name__ == '__main__':
    from tika import parser #pip install tika REQUIERE JDK

import warnings
import os
import unidecode #pip install unidecode
import time
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel

Para nuestro proyecto bajamos los PDF de los planes de cada materia de la carrera de computación.  
Esta sección parsea los datos de los pdf a un formato más manejable, archivos de texto (.txt).  
Estos estan en la carpeta "TXT/"

**NOTA**: no es necesario correr la celda de abajo ya que en el proyecto que se envio ya convertimos los pdfs a txt


In [3]:
#no es necesario correr esta sección ya que en el proyecto que se envio ya convertimos los pdfs a txt

directory = "pdf/"

if __name__ == '__main__':
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"): 
            print(filename)
            raw = parser.from_file(os.path.join(directory, filename))
            #print(raw['content'])
            lines = raw['content']

            lines = unidecode.unidecode(lines[77:])
            #print(lines)
            f = open(os.path.join("TXT/", filename[:-3]+ "txt"), 'w')
            f.write(lines)
            f.close()
            time.sleep(.9)
            continue
        else:
            continue


1089.pdf


2022-01-31 20:18:15,609 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


1323.pdf
1326.pdf
1425.pdf
1427.pdf
1428.pdf
1429.pdf
1506.pdf
1613.pdf
1827.pdf
1828.pdf
1829.pdf
249.pdf
269.pdf
277.pdf
291.pdf
351.pdf
442.pdf
446.pdf
447.pdf
5.pdf
625.pdf
626.pdf
668.pdf
669.pdf
670.pdf
671.pdf
672.pdf
673.pdf
674.pdf
676.pdf
714.pdf
759.pdf
760.pdf
764.pdf
765.pdf
777.pdf
779.pdf
783.pdf
784.pdf
785.pdf
786.pdf
787.pdf
788.pdf
789.pdf
790.pdf
791.pdf
792.pdf
793.pdf
794.pdf
803.pdf
805.pdf
809.pdf
817.pdf
819.pdf
826.pdf
840.pdf
1.pdf
10.pdf
1123.pdf
1125.pdf
1126.pdf
1222.pdf
1226.pdf
1707.pdf
2.pdf
2041.pdf
259.pdf
398.pdf
399.pdf
422.pdf
6.pdf
654.pdf
655.pdf
656.pdf
657.pdf
658.pdf
659.pdf
660.pdf
661.pdf
662.pdf
663.pdf
664.pdf
665.pdf
666.pdf
667.pdf
7.pdf
8.pdf
801.pdf
1532.pdf
1533.pdf
1534.pdf
1535.pdf
1536.pdf
575.pdf
608.pdf
621.pdf
632.pdf
633.pdf
634.pdf
675.pdf
677.pdf
678.pdf
679.pdf
713.pdf
771.pdf
772.pdf
773.pdf
774.pdf
781.pdf
782.pdf
814.pdf
820.pdf
903.pdf


# Aqui empieza el main

El principal problema para la creacion de nuestro recomendador es el Natural Language Processing o procesamiento de lenguaje natural .  
Para extraer caracteristicas y similitudes entre los documentos, vamos a calcular los vectores de palabras de cada uno de estos.  
Los vectores de palabras son representaciones vectoriales de las palabras de un documento.

Para crearlos vamos a utilizar el Term Frequency-Inverse Document Frequency (TF-IDF). Esto nos devolvera una matriz donde cada columna represente una palabra en nuestro vocabulario, y cada columna representara un plan de estudios.


In [4]:
import os
import unidecode #pip install unidecode
import time
import warnings
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel



# Primero crearemos un diccionario con todos los documentos para poder pasarlos al TF-IDF Vectorizer.


directory = "TXT/"

materias = {}
#print("hola")
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        #print(filename) 
        f = open(os.path.join(directory, filename), 'r')
        materias[filename[:-4]] = f.read()

#print(materias['1.txt'])


Para nuestros fines, al tener muchas palabras recurrentes entre los documentos que no sirven para los fines de calculos de similitudes, tenemos que agregarlas a las stopwords para que no sean consideradas para los calculos de similitudes. Son palabras que se encontraban en la mayoria de nuestros archivos como: "Semestre","Asignatura","examenes",etc.

In [5]:
"""
Este metodo crea el objeto TF-IDF Vectorizer y regresa la matriz de similitudes para poder usarla para crear nuestras sugerencias. 
Recibe como parametro los intereses del alumno con base en los cuales haremos las sugerencias. 
"""
def train(intereses):
    
    #Palabras que no consideraremos.
    stop_words = stopwords.words('spanish')
    my_stop_words = ["CARRERA", "MATEMATICO", "SERIACION", "INDICATIVA", "ANTECEDENTE", "MODALIDAD",
                     "CURSO", "CARACTER", "TEORICAS", "PRACTICAS", "CREDITOS", "HORAS", "SEMANA", "SEMESTRE", "CLAVE",
                     "FACULTAD", "CIENCIAS", "OBJETIVO", "BIBLIOGRAFIA", "BASICA", "COMPLEMENTARIA", "SUGERENCIA",
                     "PARA", "LA", "EVALUACION", "DE", "LA", "ASIGNATURA", "Ademas", "de", "las", "calificaciones",
                     "en", "examenes","examen", "y", "tareas", "se", "tomara", "en", "cuenta", "la", "participacion", "del", "alumno",
                     "PERFIL", "PROFESIOGRAFICO", "Matematico", "fisico", "actuario", "o", "licenciado", "en", "ciencias",
                     "de", "la", "computacion", "especialista", "en", "el", "area", "de", "la", "asignatura", "a", "juicio",
                     "del", "comite", "de", "asignacion", "de", "cursos", "UNIDADES", "TEMATICAS", "SERIACION",
                     "INDICATIVA", "ANTECEDENTE", "SUBSECUENTE", "optativo", "Optativas", "VERSIDAD", "Computacion", "el",
                     "plan", "y", "programa", "de", "estudios", "de", "la", "licenciatura", "en", "Ciencias", "Computacion",
                     ">Ciencias",
                     ]
    stop_words.append(my_stop_words)
    materias[0] = intereses
    
    data = pd.Series(materias).to_frame()
    #type(data)
    data = data.rename(columns={0: 'Planes'})
    #data["index"] = pd.to_numeric(data["index"])
    data.index = data.index.map(int)
    data.sort_index(inplace=True)

    #print(data.columns)
    #print(data.index)


    #Creamos un objeto TF-IDF Vectorizer. Y le decimos que palabras ignorar, en este caso, nuestra version extendida de las stop_words. 
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        tfidf = TfidfVectorizer(stop_words)

    #Creamos la matriz TF-IDF ajustando y transformando los datos
    #print(data['Planes'])
    tfidf_matrix = tfidf.fit_transform(data['Planes'])

    #Imprimimos las dimensiones de nuestra matriz
    tfidf_matrix.shape

    # Con get_feature_names podemos imprimir las palabras encontradas en todos los archivos 
    # ordenadas segun las ocurrencias usamos esta linea para encontrar palabras que agregar a nuestras stopwords. 
    # las palabras que se repetian mucho sin contribuir a los temas de los planes. como (matematicas,semestre,carrera,etc)
    tfidf.get_feature_names()[2990:3007]


    # Finalmente calculamos la matriz coseno de similaridades.
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    (cosine_sim.shape)
    #print(cosine_sim[1])
    
    return data, cosine_sim
    


In [6]:
"""
Este metodo manda a llamar al metodo train, el cual creara la matriz de similitudes que usaremos para nuestras sugerencias. 

Recibe como parametro los intereses del alumno con base en los cuales haremos las sugerencias. 

Regresa las 10 claves de materia mas similares a los gustos del estudiante. 
"""


def get_recommendations(intereses):
    # Obtenemos los indices con la matriz de similitudes.
    data, matriz_similitudes = train(intereses)

    # Tomamos la similitud de cada documento con nuestra lista de intereses.
    # Recordemos que nuestra lista de intereses es el indice 0 de la matriz, asi lo hicismo en train()
    sim_scores = list(enumerate(matriz_similitudes[0]))
    #print(sim_scores)

    # Ordenamos los planes basados en las puntuaciones de similitud.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #print(sim_scores)

    # Tomamos las puntuaciones de los primeros 10 mas similares.
    sim_scores = sim_scores[0:11]

    # Buscamos los indices de estos primeros 10, para saber la clave de materias a la que corresponden
    recomendaciones = [i[0] for i in sim_scores]
    #print(recomendaciones)

    # Regresamos las 10 claves de materias mas similares
    return list(data.iloc[recomendaciones].index[1:])

## Ejemplos (Clave materias)

In [7]:
recomendacion = get_recommendations("Ecuaciones diferenciales")
recomendacion

[1429, 1089, 786, 1707, 5, 7, 784, 781, 764, 777]

In [8]:
recomendacion = get_recommendations("Analisis Númerico criptografia")
recomendacion


[669, 10, 1828, 1532, 801, 1089, 817, 1707, 399, 1533]

In [9]:
get_recommendations('Inteligencia Artificial')


[608, 664, 773, 676, 903, 781, 819, 1532, 1323, 666]

## Ejemplos (Nombre Materias)

** Para usar el recomendador, la sintaxis sera 

In [10]:
#Para mejorar la facilidad de los resultados, vamos a pasar de la clave de materias a el nombre de la materia. 

def recomendaciones(intereses):
    listaClaves = get_recommendations(intereses)


    data = pd.read_csv("claves.csv")
    lista= []
    for clave in listaClaves:
        lista.append(data[data['CLAVE'] == clave][["CLAVE","NOMBRE","SEMESTRE"]])

    data = pd.concat(lista)
    data.reset_index(drop=True, inplace=True)
    return data


In [11]:
recomendaciones('Inteligencia Artificial')


,CLAVE,NOMBRE,SEMESTRE
0,608,Inteligencia Artificial,6
1,773,Reconocimiento de Patrones y Aprendizaje Autom...,Optativa
2,676,Programación Declarativa,Optativa
3,903,Robótica Móvil,Optativa
4,781,Redes Neuronales,Optativa
5,819,Lingüística Computacional,Optativa
6,1532,Análisis de Algoritmos,5
7,1323,Modelado y Programación,3


In [12]:
recomendaciones("Analisis Númerico criptografia")


,CLAVE,NOMBRE,SEMESTRE
0,669,Introducción a la Criptografía,Optativa
1,1828,Criptografí­a y Seguridad,8
2,1532,Análisis de Algoritmos,5
3,1089,Tecnologí­as para Desarrollos en Internet,Optativa
4,817,Compiladores,7
5,1533,Computación Distribuida,5


In [13]:
recomendaciones("Ecuaciones diferenciales")


,CLAVE,NOMBRE,SEMESTRE
0,1429,Matemáticas para las Ciencias Aplicadas IV,4
1,1089,Tecnologí­as para Desarrollos en Internet,Optativa
2,786,Sistemas Dinámicos Computacionales I,Optativa
3,5,Álgebra Lineal I,3
4,784,Seminario de Ciencias de la Computación B,Optativa
5,781,Redes Neuronales,Optativa
6,764,Teorí­a de los Números I,Optativa
7,777,Teorí­a de los Números II,Optativa
